```
conda create -y -n scraping python=3.8
conda activate scraping
pip install requests yt-dlp ffmpeg-python
```

In [1]:
import requests

access_token = open('.access_token').read()

headers = {"Authorization": f"Bearer {access_token}"}
page = 1
all_videos = []
while True:
    print('Requesting page', page)
    response = requests.get(
        f"https://api.vimeo.com/users/inboundfilmsproductions/videos?page={page}&per_page=100", headers=headers)
    if response.status_code != 200:
        print('Finished: code', response.status_code)
        break
    videos = response.json()["data"]
    if not videos:
        print('Finished: no more videos')
        break
    print(f'Added: {len(videos)} videos for {len(all_videos)} total')
    all_videos.extend(videos)
    page += 1

Requesting page 1
Added: 97 videos for 0 total
Requesting page 2
Added: 98 videos for 97 total
Requesting page 3
Added: 98 videos for 195 total
Requesting page 4
Added: 85 videos for 293 total
Requesting page 5
Added: 95 videos for 378 total
Requesting page 6
Added: 100 videos for 473 total
Requesting page 7
Added: 100 videos for 573 total
Requesting page 8
Added: 96 videos for 673 total
Requesting page 9
Added: 96 videos for 769 total
Requesting page 10
Added: 88 videos for 865 total
Requesting page 11
Added: 100 videos for 953 total
Requesting page 12
Added: 96 videos for 1053 total
Requesting page 13
Added: 100 videos for 1149 total
Requesting page 14
Added: 100 videos for 1249 total
Requesting page 15
Added: 100 videos for 1349 total
Requesting page 16
Added: 100 videos for 1449 total
Requesting page 17
Added: 77 videos for 1549 total
Requesting page 18
Finished: code 400


In [2]:
import json
with open('videos.json', 'w') as f:
    json.dump(all_videos, f, indent=2)

In [3]:
video_urls = []
collected = []
for video in sorted(all_videos, key=lambda e: e['duration']):
    n = video['name'].lower()
    if not ('police' in n or 'lapd' in n):
        continue
    video_urls.append(video["link"])
    collected.append({
        'name': video['name'],
        'duration': video['duration'],
        'link': video['link'],
        'thumbnail': video["pictures"]["sizes"][-1]["link"],
        'created_time': video['created_time'],
        'modified_time': video['modified_time'],
        'release_time': video['release_time'],
    })

In [4]:
import pandas as pd
pd.DataFrame(collected).to_csv('lapd-videos.csv', index=False)
with open('video-urls.txt', 'w') as f:
    f.write('\n'.join(video_urls))

In [5]:
!mkdir -p videos && cd videos && yt-dlp --batch-file ../video-urls.txt --download-sections "*0:00-0:10"

[vimeo] Extracting URL: https://vimeo.com/820913028
[vimeo] 820913028: Downloading webpage
[vimeo] 820913028: Downloading JSON metadata
[vimeo] 820913028: Downloading JSON metadata
[vimeo] 820913028: Checking Original video URL
[vimeo] 820913028: Downloading akfire_interconnect_quic m3u8 information
[vimeo] 820913028: Downloading akfire_interconnect_quic m3u8 information
[vimeo] 820913028: Downloading fastly_skyfire m3u8 information
[vimeo] 820913028: Downloading fastly_skyfire m3u8 information
[vimeo] 820913028: Downloading akfire_interconnect_quic MPD information
[vimeo] 820913028: Downloading akfire_interconnect_quic MPD information
[vimeo] 820913028: Downloading fastly_skyfire MPD information
[vimeo] 820913028: Downloading fastly_skyfire MPD information
[info] 820913028: Downloading 1 format(s): Original
[info] 820913028: Downloading 1 time ranges: 0.0-10.0
[download] Edwin Romero - LAPD Graduation [820913028].mp4 has already been downloaded
[download] 100% of   29.51MiB
[vimeo] Ex